In [ ]:
import requests as r
import pandas as pd
import shutil
import os
import chardet
from zipfile import ZipFile

dev = False

tmp_dir = 'julienne_data/'
start = 2012
end = 2018
ipeds_url = 'https://nces.ed.gov/ipeds/datacenter/data/'
ipeds_tuples = [('HD', '', ['UNITID', 'INSTNM', 'IALIAS', 'ADDR', 'CITY', 'STABBR', 'ZIP', 'FIPS', 'OBEREG', 'OPEFLAG', 'WEBADDR', 'SECTOR', 'ICLEVEL', 'CONTROL', 'HLOFFER', 'UGOFFER', 'GROFFER', 'HDEGOFR1', 'DEGGRANT', 'HBCU', 'OPENPUBL', 'ACT', 'NEWID', 'DEATHYR', 'CYACTIVE', 'PSET4FLG', 'INSTCAT', 'C15BASIC', 'C15IPUG', 'C15IPGRD', 'C15UGPRF', 'C15ENPRF', 'C15SZSET', 'CCBASIC', 'CARNEGIE', 'LANDGRNT', 'INSTSIZE', 'F1SYSTYP', 'F1SYSNAM', 'F1SYSCOD', 'COUNTYCD', 'COUNTYNM', 'CNGDSTCD', 'LONGITUD', 'LATITUDE']),
                ('IC', '', ['UNITID', 'CNTLAFFI', 'PUBPRIME', 'PUBSECON', 'RELAFFIL', 'DSTNUGC', 'DSTNUGP', 'DSTNUGN', 'DSTNGC', 'DSTNGP', 'DSTNGN', 'DISTCRS', 'DISTPGS', 'DSTNCED1', 'DSTNCED2', 'DSTNCED3', 'DISTNCED', 'ALLONCAM']),
                ('IC', '_AY', ['UNITID', 'TUITION1', 'FEE1', 'HRCHG1', 'TUITION2', 'FEE2', 'HRCHG2', 'TUITION3', 'FEE3', 'HRCHG3', 'TUITION5', 'FEE5', 'HRCHG5', 'TUITION6', 'FEE6', 'HRCHG6', 'TUITION7', 'FEE7', 'HRCHG7']),
                ('EF', 'A_DIST', ['UNITID', 'EFDELEV', 'EFDETOT', 'EFDEEXC', 'EFDESOM', 'EFDENON', 'EFDEEX1', 'EFDEEX2', 'EFDEEX3', 'EFDEEX4', 'EFDEEX5']),
                ('SAL', '_IS', ['UNITID', 'ARANK', 'SAINSTT', 'SAINSTM', 'SAINSTW', 'SA_9MCT', 'SA_9MCM', 'SA_9MCW', 'SATOTLT', 'SATOTLM', 'SATOTLW', 'SAOUTLT', 'SAOUTLM', 'SAOUTLW', 'SAEQ9OT', 'SAEQ9OM', 'SAEQ9OW', 'SAEQ9AT', 'SAEQ9AM', 'SAEQ9AW']),
                ('OM', '', ['UNITID', 'OMCHRT', 'OMACHRT', 'OMAWDP8', 'OMENRTP', 'OMENRYP', 'OMENRAP', 'OMENRUP'])
             ]

if os.path.isdir(tmp_dir) and not dev:
    shutil.rmtree(tmp_dir)

if not (dev and os.path.isdir(tmp_dir)):
    os.mkdir(tmp_dir)


In [ ]:
# Dataframes to be populated and merged per year: hd_df, ic_df, icay_df, efdist_df, salis_df, om_df

for year in range(start, end):
    df = {}
    for prefix, suffix, cols in ipeds_tuples:
        
        # Skip missing datasets
        if prefix == "OM" and year < 2015:
            continue
            
        # Download and process the existing data
        else:
            url = ipeds_url + prefix + str(year) + suffix + '.zip'
            zip = tmp_dir + prefix + str(year) + suffix + '.zip'
            file = prefix.lower() + str(year) + suffix.lower() + '.csv'
            stream = r.get(url, stream=True)    
            print(str(stream.status_code) + '\t' + url)
            if stream.status_code == 200:
                with open(zip, 'wb') as out_file:
                    shutil.copyfileobj(stream.raw, out_file)
                z = ZipFile(zip).open(file)
                ZipFile(zip).extract(file, tmp_dir)
                enc_result = chardet.detect(z.read())
                tmp_df = pd.read_csv(tmp_dir + '/' + file, encoding=enc_result['encoding'])
            else:
                sys.exit('Failed to retrieve: ' + url)

        # Eliminate and calculate columns here        
        # Set null values for missing but expected columns
        for col in cols:
            if col not in tmp_df.columns.values:
                tmp_df[col] = 'Null'
        # Drop other columns
        tmp_df = tmp_df[cols]
        # First the easy ones: HD, IC, IC_AY - just take the columns we need
        if (prefix, suffix) in [('HD', ''), ('IC', ''), ('IC', '_AY')]:
            df[prefix + suffix] = tmp_df[cols]
        elif (prefix, suffix) == ('EF', 'A_DIST'):
            # Transpose rows to cols on EFDELEV (drop and recalculate)
            tr = 'EFDELEV'
            di = {
                1  : '_All',
                2  : '_Under',
                3  : '_Degree',
                11 : '_Non',
                12 : '_Grad'
            }
            tmp_df[tr] = tmp_df[tr].map(di)
            tmp_df = tmp_df.set_index(['UNITID', tr]).unstack().reset_index()
            tmp_df.columns = tmp_df.columns.map(''.join)
            for col in tmp_df.columns:
                tmp_df[col] = tmp_df[col].fillna('Null')
        elif (prefix, suffix) == ('SAL', '_IS'):
            # Transpose rows to cols on ARANK
            tr = 'ARANK'
            di = {
                1 : '_Professor',
                2 : '_Associate',
                3 : '_Assistant',
                4 : '_Instructor',
                5 : '_Lecturer',
                6 : '_NoRank',
                7 : '_All'
            }
            tmp_df[tr] = tmp_df[tr].map(di)
            tmp_df = tmp_df.set_index(['UNITID', tr]).unstack().reset_index()
            tmp_df.columns = tmp_df.columns.map(''.join)
            for col in tmp_df.columns:
                tmp_df[col] = tmp_df[col].fillna('Null')
        elif (prefix, suffix) == ('OM', ''):
            # Transpose rows to cols on OMCHRT
            # Remove unwanted values
#             tmp_df = tmp_df[~tmp_df['OMCHRT'].isin([30, 31, 32, 40, 41, 42])]
            continue
#             tr = 'OMCHRT'
#             di = {
#                 10 : '_Total',
#                 11 : '_TotalPell',
#                 12 : '_TotalNonpell',
#                 20 : '_FirstFullTotal',
#                 21 : '_FirstFullPell',
#                 22 : '_FirstFullNonpell',
#                 50 : '_FirstPartTotal',
#                 51 : '_FirstPartPell',
#                 52 : '_FirstPartNonpell'
#             }
#             tmp_df[tr] = tmp_df[tr].map(di)
#             tmp_df = tmp_df.set_index(['UNITID', tr]).unstack().reset_index()
#             tmp_df.columns = tmp_df.columns.map(''.join)
#             for col in tmp_df.columns:
#                 tmp_df[col] = tmp_df[col].fillna(0)
        else:
            sys.exit("Don't know what to do with this file" + file)

    
    # Append columns into working year (add year)
    
    
    # And initiate or merge to master frame
#     if year == start:
#         ipeds = __MERGED_DF__
#         ipeds['YEAR'] = year
#     else
#         pd.concat(ipeds, merged, ignore_index=True)
    
# Recode categorical values      

In [ ]:
# tmp_df.head()

In [ ]:
if not dev:
    shutil.rmtree(tmp_dir)